In [1]:
from numpy.random import default_rng
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from collections import Counter
from collections import deque
import matplotlib.cm as cm
import networkx as nx
import numpy as np
import random
import time

rng = default_rng()

In [2]:
# Function that executes a tensor product over an ordered list of objects(arrays or matrices)
# specified by the argument "k".
def tensor(k):
    t = k[0]
    i = 1
    while i < len(k) :
        t = np.kron(t,k[i])
        i+=1
    return t

def graph_to_hamiltonian(G,n): 
    H = np.zeros((2**n), dtype = 'float64') 
    Z = np.array([1,-1],dtype = 'float64')
    for i in range(n):
        j = i+1
        while j<n: 
            k = [[1,1]]*n 
            k = np.array(k,dtype = 'float64')
            if G[i][j] !=0: 
                k[i] = Z
                k[j] = Z
                H+= tensor(k)*G[i][j] 
            j+=1
    return H

def ZZ(G, i, j, n): 
    H = np.zeros((2**n), dtype = 'float64') 
    Z = np.array([1,-1],dtype = 'float64')
    k = [[1,1]]*n 
    k = np.array(k,dtype = 'float64')    
    if G[i][j] != 0: 
        k[i] = Z
        k[j] = Z
        H+= tensor(k)*G[i][j] 
    return H

def compute_subgraph_for_edge(G, u, v):
    v_sub = nx.Graph()
    v_sub.add_edge(u, v)
    for n in G.neighbors(u):
        v_sub.add_edge(u, n)
    for n in G.neighbors(v):
        v_sub.add_edge(v, n)
    old_nodes = list(v_sub.nodes())
    mapping = {old: new for new, old in enumerate(old_nodes)}
    v_sub = nx.relabel_nodes(v_sub, mapping)
    new_u = mapping[u]
    new_v = mapping[v]
    n_sub = len(list(v_sub.nodes()))
    return v_sub, n_sub, new_u, new_v

# Generates a random k-sat instance
def k_sat_instance (n,k,m):
    instance=np.array([])
    for mind in range(m):
        clause = rng.choice(n, size=k, replace=False)
        instance=np.append(instance,clause)
    instance=(instance+1) *((-1)**np.random.randint(2,size=k*m))
    return  instance.reshape(m,k)

# Converts the k-sat instance into a Hamiltonian
def H_sat(inst,n):
    I=[1,1]
    k0=[1,0]
    k1=[0,1]
    inst_tp=0
    for mind in range(inst.shape[0]):
        clause=inst[mind]
        clause_list=[I]*n
        for lind in range(inst.shape[1]):
            if clause[lind]<0:
                clause_list[int(np.abs(clause[lind])-1)]=k1
            else:
                clause_list[int(np.abs(clause[lind])-1)]=k0
        clause_tp=tensor(clause_list)
        inst_tp=inst_tp+clause_tp
    return np.array(inst_tp)

In [146]:
def sample_from_state(state, n, shots=100, return_probs=False):
    state = state.reshape(-1)
    probs = np.abs(state) ** 2
    indices = np.random.choice(len(probs), size=shots, p=probs)
    most_common_index = Counter(indices).most_common(1)
    bitstring = format(most_common_index[0][0], f"0{n}b")
    if return_probs:
        return bitstring, probs
    return bitstring

def find_assignment(G, constraints):
    assignment = {}
    for start in G.nodes:
        if start in assignment:
            continue
        assignment[start] = 0
        queue = deque([start])
        while queue:
            u = queue.popleft()
            for v in G.neighbors(u):
                if (u, v) in constraints:
                    sign = constraints[(u, v)]
                elif (v, u) in constraints:
                    sign = constraints[(v, u)]
                else:
                    continue
                expected = assignment[u] if sign == 1 else 1 - assignment[u]
                if v not in assignment:
                    assignment[v] = expected
                    queue.append(v)
                else:
                    if assignment[v] != expected:
                        raise ValueError("Constraints are inconsistent")
    return assignment

`U_gamma`: $$|\psi\rangle e^{-i\gamma H_{C}}=\left(\begin{array}{c}
\alpha_{0}\\
\alpha_{1}\\
\alpha_{2}\\
\alpha_{3}
\end{array}\right)e^{-i\gamma\left(\begin{array}{c}
\beta_{0}\\
\beta_{1}\\
\beta_{2}\\
\beta_{3}
\end{array}\right)}=\left(\begin{array}{c}
\alpha_{0}e^{-i\gamma\beta_{0}}\\
\alpha_{1}e^{-i\gamma\beta_{1}}\\
\alpha_{2}e^{-i\gamma\beta_{2}}\\
\alpha_{3}e^{-i\gamma\beta_{3}}
\end{array}\right)$$

`new_mixerX`: 
Applying $e^{-i\beta X}$ to the qubit $i$ swaps amplitudes of basis states whose binary labels differ only at bit $i$. This method precomputes index permutations to later apply Pauli-X efficiently (via array indexing).


`V_beta`:
If the number of qubits is $n=1$: $$|\psi\rangle=\cos\beta|\psi\rangle-i\sin\beta X_{i}|\psi\rangle $$

If $n>1$, for each qubit $i=0,...,n-1$ update the state as follows: $$|\psi^{(i)}\rangle=\cos\beta|\psi^{(i-1)}\rangle-i\sin\beta X_{i}|\psi^{(i-1)}\rangle)$$

In each iteration we apply X to qubit $i$ and reindex the state vector



In [3]:
class QAOA:
    def __init__(self,depth,H):     # Class initialization. Arguments are "depth",
                                            # and a Diagonal Hamiltonian,"H".
        self.H = H 
        self.n = int(np.log2(int(len(self.H)))) # Calculates the number of qubits.

        #______________________________________________________________________________________________________
        self.X = self.new_mixerX()        # Executes a sequence of array manipulations to encapsulate the
                                                # effect of standard one body driver hamiltonian, \Sum \sigma_x,
                                                # acting on any state.
        #______________________________________________________________________________________________________


        self.min = min(self.H)                  # Calculates minimum of the Hamiltonain, Ground state energy.

        self.deg = len(self.H[self.H == self.min]) # Calculates the degeneracy of Ground states.
        self.p = depth                             # Standard qaoa depth written as "p".

        #______________________________________________________________________________________________________

                    # The sequence of array manipulations that return action of the driver,
                    # in terms of permutation indices.

    def new_mixerX(self):
        def split(x,k):
            return x.reshape((2**k,-1))
        def sym_swap(x):
            return np.asarray([x[-1],x[-2],x[1],x[0]])

        n = self.n
        x_list = []
        t1 = np.asarray([np.arange(2**(n-1),2**n),np.arange(0,2**(n-1))])
        t1 = t1.flatten()
        x_list.append(t1.flatten())
        t2 = t1.reshape(4,-1)
        t3 = sym_swap(t2)
        t1 = t3.flatten()
        x_list.append(t1)

        k = 1
        while k < (n-1):
            t2 = split(t1,k)
            t2 = np.asarray(t2)
            t1=[]
            for y in t2:
                t3 = y.reshape((4,-1))
                t4 = sym_swap(t3)                
                t1.append(t4.flatten())
            t1 = np.asarray(t1)
            t1 = t1.flatten()
            x_list.append(t1)
            k+=1

        return x_list
    #__________________________________________________________________________________________________________


    def U_gamma(self,angle,state):       # applies exp{-i\gamma H_z}, here as "U_gamma", on a "state".
        t = -1j*angle
        H_new = self.H.reshape(2**self.n,1)
        
        state = state*np.exp(t*self.H.reshape(2**self.n,1))

        return state

    
    def V_beta(self,angle,state):        # applies exp{-i\beta H_x}, here as "V_beta", on a "state".
        c = np.cos(angle)
        s = np.sin(angle)

        for i in range(self.n):
            t = self.X[i]
            st = state[t]
            state = c*state + (-1j*s*st)

        return state

    #__________________________________________________________________________________________________________

                        # This step creates the qaoa_ansatz w.r.t to "angles" that are passed.
                        # "angles" are passed as [gamma_1,gamma_2,...,gamma_p,beta_1,beta2,....beta_p].

    def qaoa_ansatz(self, angles):
        
        state = np.ones((2**self.n,1),dtype = 'complex128')*(1/np.sqrt(2**self.n))
        
        for i in range(self.p):
            state = self.U_gamma(angles[i],state)
            state = self.V_beta(angles[self.p + i],state)

        return state


    #__________________________________________________________________________________________________________


    def expectation(self,angles):   # Calculates expected value of the Hamiltonian w.r.t qaoa_ansatz state,
                                           # defined by the specific choice of "angles".
                                                                   
        state = self.qaoa_ansatz(angles)
        ex = np.vdot(state,state*(self.H).reshape((2**self.n,1)))
        return  np.real(ex)
    

    def overlap(self,state):        # Calculates ground state overlap for any "state",
                                                        # passed to it. Usually the final state  or "f_state" returned,
                                                        # after optimization.

        g_ener = min(self.H)
        olap = 0
        for i in range(len(self.H)):
            if self.H[i] == g_ener:
                olap+= np.absolute(state[i])**2

        return olap

   #__________________________________________________________________________________________________________

                    # Main execution of the algorithm.
                    # 1) Create "initial_angles", this would be the guess or
                    #    starting point for the optimizer.
                    # 2) Optimizer "L-BFGS-B" then takes "initial_angles" and calls "expectation".
                    # 3) "expectation" then returns a number and the optimizer tries to minimize this,
                    #     by doing finite differences. Thermination returns optimized angles,
                    #     stored here as "res.x".
                    # 4) Treating the optimized angles as being global minima for "expectation",
                    #    we calculate and store (as class attributes) the qaoa energy, here as "q_energy",
                    #    energy error, here as "q_error",
                    #    ground state overlap, here as "olap"
                    #    and also the optimal state, here as "f_state"

    def run(self):
        initial_angles=[]
        bds= [(0,2*np.pi+0.1)]*self.p + [(0,1*np.pi+0.1)]*self.p
        for i in range(2*self.p):
            if i < self.p:
                initial_angles.append(random.uniform(0,2*np.pi))
            else:
                initial_angles.append(random.uniform(0,np.pi))

        # start = time.perf_counter()
        res = minimize(self.expectation,initial_angles,method='L-BFGS-B', jac=None, bounds=bds, options={'maxiter': 1000})
     
        # elapsed = time.perf_counter() - start
        # res = differential_evolution(self.expectation, bds,maxiter=100, callback=None, disp=False, init='latinhypercube',workers=-1)
    
        self.q_energy = self.expectation(res.x)
        self.q_error = self.q_energy - self.min
        self.f_state = self.qaoa_ansatz(res.x)
        self.olap = self.overlap(self.f_state)[0]
        self.angles = res.x
        self.best_bitstring = np.binary_repr(int(np.argmax(np.abs(self.f_state))), width=self.n)
        
    
        
     #__________________________________________________________________________________________________________


The Light Cones method leverages on the following property of the expectation function:
$$\langle\psi|H|\psi\rangle=\underset{(u,v)\in E}{\sum}\langle\psi|Z_{u}Z_{v}|\psi\rangle$$

It involves decomposing the $|V|$ qubits circuit into $|E|$ smaller sub-circuits:
- Assuming $p=1$, for each edge $(u,v)$, we have a sub-circuit with $deg(u)+deg(v)$ qubits

We decompose, we measure and reconstruct the expectations.

In practice, every time we want to compute the expectation value of the Hamiltonian, we iterate over the edges and:
- compute the subgraph
- compute the small Hamiltonian starting from the subgraph
- constrcut the ansatz that applies the small Hamiltonian
- measure 

In [238]:
class LightCone:
    def __init__(self, G, u, v, p):
        self.u, self.v = u, v
        self.v_sub, self.n_sub, self.new_u, self.new_v = compute_subgraph_for_edge(G, u, v)
        self.H = graph_to_hamiltonian(nx.to_numpy_array(self.v_sub), self.n_sub)
        self.Z_uZ_v = ZZ(nx.to_numpy_array(self.v_sub), self.new_u, self.new_v, self.n_sub)
        self.Q = QAOA(p, self.H)
        
    def expectation(self, angles):
        state = self.Q.qaoa_ansatz(angles)
        col_shape = (2**self.Q.n, 1)
        ex = np.vdot(state, state * self.Z_uZ_v.reshape(col_shape))
        self.expectation = np.real(ex)
        return np.real(ex)

class Simulation:
    def __init__(self, G, p):
        self.G = G
        self.p = p
        self.light_cones = []
        for u, v in self.G.edges:
            self.light_cones.append(LightCone(G, u, v, p))

    def sum_of_expectations(self, angles, return_energy=True):
        total_energy = 0.0
        for lc in self.light_cones:
            total_energy += lc.expectation(angles)
        if return_energy:
            return total_energy
    
    def compute_bitstring(self, angles):
        self.sum_of_expectations(angles, return_energy=False)
        expectations = {}
        for lc in self.light_cones:
            expectations[(lc.u, lc.v)] = np.sign(lc.expectation)
        # print(f"Correlation map: {expectations}")
        assignment = find_assignment(self.G, expectations)
        maxcut = [assignment[key] for key in sorted(assignment)]  # I don't know if sorting is actually necessary 
        return ''.join(map(str, maxcut))
        
    def run(self): 
        initial_angles=[]
        bds= [(0,2*np.pi+0.1)]*self.p + [(0,1*np.pi+0.1)]*self.p
        for i in range(2*self.p):
            if i < self.p:
                initial_angles.append(random.uniform(0,2*np.pi))
            else:
                initial_angles.append(random.uniform(0,np.pi))
        res = minimize(self.sum_of_expectations,initial_angles,method='L-BFGS-B', jac=None, bounds=bds, options={'maxiter': 1000})
        self.angles = res.x
            

Before removal: $H_{n}=\underset{(p,q)\in E}{\sum}J_{p,q}Z_{p}Z_{q}$

Removal of the qubit `q`:

1. Replace $Z_{q}=sgn(M_{p,q})Z_{p}$ in the Hamiltonian
2. The Hamiltonian becomes: $$H_{n-1}=J_{p,q}sgn(M_{p,q})I+\underset{(i,j)\in E\wedge(i,j)\neq q}{\sum}J_{i,j}Z_{i}Z_{j}+\underset{(q,k)\in E\wedge(k\neq p,q)}{\sum}J_{q,k}sgn(M_{p,q})Z_{p}Z_{k}$$

In [235]:
def reduce_hamiltonian(G, n, p, q, sgn):
    dim = 2**(n-1)
    H = np.zeros((dim), dtype='float64')
    Z = np.array([1, -1], dtype='float64')

    def new_index(i):
        if i < q:
            return i
        elif i > q:
            return i - 1
        else:
            raise ValueError("index q eliminated")

    for i in range(n):
        for j in range(i+1, n):
            k = [[1,1]]*(n-1) 
            k = np.array(k,dtype = 'float64')
            if G[i][j] == 0:
                continue

            # if (i == p and j == q) or (i == q and j == p):
            #     k = [1] * 2**(n-1)
            #     k = np.array(k,dtype = 'float64')  
            #     H += G[i][j] * sgn * k

            elif i != q and j != q:
                k[new_index(i)] = Z
                k[new_index(j)] = Z
                H += tensor(k) * G[i][j]

            else:
                other = j if i == q else i
                if other == p:
                    continue  

                k[new_index(p)] = Z
                k[new_index(other)] = Z
                H += tensor(k) * G[i][j] * sgn
    return H

def update_mapping_after_removal(mapping, removed_id):
    new_mapping = {}
    for new_id, original_id in mapping.items():
        if new_id < removed_id:
            new_mapping[new_id] = original_id
        elif new_id > removed_id:
            new_mapping[new_id - 1] = original_id
    return new_mapping

def remove_node(G, j, mapping):
    G_new = G.copy()
    G_new.remove_node(j)
    old_nodes = list(G_new.nodes)
    labels = {old: new for new, old in enumerate(old_nodes)}
    G_new = nx.relabel_nodes(G_new, labels)
    return update_mapping_after_removal(mapping, j), G_new

class RQAOA:
    def __init__(self, G, p):
        self.G = G
        self.p = p
        
    def expectation(self, Q, Z_iZ_j, angles):
        state = Q.qaoa_ansatz(angles)
        col_shape = (2**Q.n, 1)
        ex = np.vdot(state, state * Z_iZ_j.reshape(col_shape))
        return np.real(ex)
    
    def rqaoa(self, G, p, Q, angles, mapping, constraints, optimize=False):
        
        def is_terminal(G):
            for comp in nx.connected_components(G):
                sub = G.subgraph(comp)
                if sub.number_of_nodes() > 2:
                    return False
            return True

        def extract_terminal_constraints(G, Q, angles, mapping, constraints):
            for comp in nx.connected_components(G):
                sub = G.subgraph(comp)
                if sub.number_of_edges() == 1:
                    i, j = list(sub.edges)[0]
                    Z_iZ_j = ZZ(nx.to_numpy_array(G), i, j, len(G.nodes))
                    exp = self.expectation(Q, Z_iZ_j, angles)
                    constraints[(mapping[i], mapping[j])] = np.sign(exp)
            return constraints
    
        if is_terminal(G):
            return extract_terminal_constraints(G, Q, angles, mapping, constraints)
                        
        magnitude = {}
        for (i,j) in G.edges:
            Z_iZ_j = ZZ(nx.to_numpy_array(G), i, j, len(G.nodes))
            magnitude[(i, j)] = self.expectation(Q, Z_iZ_j, angles)
    
        (i,j), max_magn = max(magnitude.items(), key=lambda item: abs(item[1]))
        s = np.sign(max_magn) 
        constraints[(mapping[i],mapping[j])] = s
        H_new = reduce_hamiltonian(nx.to_numpy_array(G), len(G.nodes), i, j, s)    
        bds= [(0,2*np.pi+0.1)]*self.p + [(0,1*np.pi+0.1)]*self.p
        Q_new = QAOA(p, H_new) 
        
        if optimize:
            res = minimize(Q_new.expectation,angles,method='L-BFGS-B', jac=None, bounds=bds, options={'maxiter': 1000})
            angles = res.x
                    
        remove = j if G.degree[j] <= G.degree[i] else i
        mapping, G_new = remove_node(G, remove, mapping)
        return self.rqaoa(G_new, p, Q_new, angles, mapping, constraints, optimize)
    
    def compute_bitstring(self, angles, optimize):
        H = graph_to_hamiltonian(nx.to_numpy_array(self.G), len(self.G.nodes))    
        Q = QAOA(self.p, H) 
        constraints = self.rqaoa(self.G, self.p, Q, angles, {i: i for i in list(self.G.nodes)}, {}, optimize=optimize) 
        # print(f"Correlation map: {constraints}")
        assignment = find_assignment(self.G, constraints)
        maxcut = [assignment[key] for key in sorted(assignment)]
        return  ''.join(map(str, maxcut))
      
    def run(self):
        initial_angles=[]
        for i in range(2*self.p):
            if i < self.p:
                initial_angles.append(random.uniform(0,2*np.pi))
            else:
                initial_angles.append(random.uniform(0,np.pi))
        H = graph_to_hamiltonian(nx.to_numpy_array(self.G), len(self.G.nodes))    
        Q = QAOA(self.p, H) 
        return self.rqaoa(self.G, self.p, Q, initial_angles, {i: i for i in list(self.G.nodes)}, {}, optimize=True) 
    
    
    

In [265]:
def is_valid(x, atol=1e-9):
    return not np.isclose(x, 0.0, atol=atol)

expected = ['0101', '0111111', '111111110', '00001111']

p = 1  
n_iterations = 100
G1 = nx.Graph()
G1.add_edges_from([(0,1),(1,2),(2,3),(3,0)])

G2 = nx.Graph()
G2.add_edges_from([(0,1),(0,2),(0,3),(0,4),(0,5),(0,6)])

G3 = nx.Graph()
G3.add_edges_from([(8,0),(8,1),(8,2),(8,3),(8,4),(8,5),(8,6),(8,7)])

G4 = nx.Graph()
G4.add_edges_from([(0,4),(0,5),(0,6),(0,7),
                  (1,4),(1,5),(1,6),(1,7),
                  (2,4),(2,5),(2,6),(2,7),
                  (3,4),(3,5),(3,6),(3,7)])

graphs = []
graphs.append(G1)
graphs.append(G2)
graphs.append(G3)
graphs.append(G4)

for G, expected_result in zip(graphs, expected):
    full_hamiltonian_res = {}
    light_cones_res = {}
    rqaoa_res = {}
    nodes = list(G.nodes)
        
    for _ in range(n_iterations):
        q_energy = 0.0
        while not is_valid(q_energy):
            H = graph_to_hamiltonian(nx.to_numpy_array(G), len(nodes))    
            Q = QAOA(p, H) 
            Q.run()
            q_energy = Q.q_energy
        
        # Full Hamiltonian
        bitstring = sample_from_state(Q.f_state, n=len(G.nodes), shots=1)
        full_hamiltonian_res[bitstring]  = full_hamiltonian_res.get(bitstring, 0) + 1
        # Light Cones 
        S = Simulation(G, p)
        bitstring = S.compute_bitstring(Q.angles)
        light_cones_res[bitstring] = light_cones_res.get(bitstring, 0) + 1
        # RQAOA 
        G_new = G.copy()
        R = RQAOA(G_new, p)
        bitstring = R.compute_bitstring(Q.angles, optimize=False)
        rqaoa_res[bitstring] = rqaoa_res.get(bitstring, 0) + 1
        
    full_hamiltonian_maxcut = max(full_hamiltonian_res.items(), key= lambda x:x[1])
    light_cones_maxcut = max(light_cones_res.items(), key= lambda x:x[1])
    rqaoa_maxcut = max(rqaoa_res.items(), key= lambda x:x[1])

    print("Full Hamiltonian")
    print(f"Expected: {expected_result} Actual: {full_hamiltonian_maxcut[0]}, Count: {full_hamiltonian_maxcut[1]}")
    print("Light Cones")
    print(f"Expected: {expected_result} Actual: {light_cones_maxcut[0]}, Count: {light_cones_maxcut[1]}")
    print("RQAOA")
    print(f"Expected: {expected_result} Actual: {rqaoa_maxcut[0]}, Count: {rqaoa_maxcut[1]}")
    print(f"====================================================")


Full Hamiltonian
Expected: 0101 Actual: 0101, Count: 20
Light Cones
Expected: 0101 Actual: 0101, Count: 100
RQAOA
Expected: 0101 Actual: 0101, Count: 55
Full Hamiltonian
Expected: 0111111 Actual: 0111111, Count: 21
Light Cones
Expected: 0111111 Actual: 0111111, Count: 100
RQAOA
Expected: 0111111 Actual: 0111111, Count: 100
Full Hamiltonian
Expected: 111111110 Actual: 011111111, Count: 10
Light Cones
Expected: 111111110 Actual: 111111110, Count: 100
RQAOA
Expected: 111111110 Actual: 100000000, Count: 62
Full Hamiltonian
Expected: 00001111 Actual: 01111000, Count: 10
Light Cones
Expected: 00001111 Actual: 00001111, Count: 99
RQAOA
Expected: 00001111 Actual: 01101000, Count: 14


In [33]:
# def generate_random_graph(num_nodes: int, num_edges: int) -> nx.Graph:
#     max_edges = num_nodes * (num_nodes - 1) // 2
#     if num_edges > max_edges:
#         raise ValueError(
#             f"Too many edges. Max edges for {num_nodes} nodes is {max_edges}."
#         )
#     G = nx.Graph()
#     G.add_nodes_from(range(num_nodes))
#     possible_edges = [(i, j) for i in range(num_nodes) for j in range(i + 1, num_nodes)]
#     chosen_edges = random.sample(possible_edges, num_edges)
#     G.add_edges_from(chosen_edges)
#     return G


# def relabel_white_black(G: nx.Graph, n: int) -> nx.Graph:
#     mapping = {}
#     for i in range(n):
#         mapping[f"w_{i}"] = i
#     for i in range(n):
#         mapping[f"b_{i}"] = n + i
#     return mapping, nx.relabel_nodes(G, mapping)


# def generate_bipartite_ring_network(n: int, p: int, q: int) -> nx.Graph:
#     if n < 2:
#         raise ValueError(f"n must be > 1, got {n}")
#     if p < 1 or p >= n / 2:
#         raise ValueError(f"p must satisfy 0 < p < n/2, got p={p} with n={n}")
#     if q < 1 or q >= n:
#         raise ValueError(f"q must satisfy 0 < q < n, got q={q} with n={n}")
#     G = nx.Graph()
#     for i in range(n):
#         G.add_node(f"b_{i}", color="black")
#         G.add_node(f"w_{i}", color="white")
#     for i in range(n):
#         for j in range(1, p + 1):
#             next_idx = (i + j) % n
#             G.add_edge(f"b_{i}", f"b_{next_idx}")
#             G.add_edge(f"w_{i}", f"w_{next_idx}")
#     for i in range(n):
#         for j in range(1, q + 1):
#             next_idx = (i + j) % n
#             G.add_edge(f"b_{i}", f"w_{next_idx}")
#     return relabel_white_black(G, n)

# def find_assignment(G, constraints):
#     assignment = {}
#     for start in G.nodes:
#         if start in assignment:
#             continue
#         assignment[start] = 0
#         queue = deque([start])
#         while queue:
#             u = queue.popleft()
#             for v in G.neighbors(u):
#                 if (u, v) in constraints:
#                     sign = constraints[(u, v)]
#                 elif (v, u) in constraints:
#                     sign = constraints[(v, u)]
#                 else:
#                     continue
#                 expected = assignment[u] if sign == 1 else 1 - assignment[u]
#                 if v not in assignment:
#                     assignment[v] = expected
#                     queue.append(v)
#                 else:
#                     if assignment[v] != expected:
#                         raise ValueError("Constraints are inconsistent")
#     return assignment


In [ ]:
# n_iterations = 200
# p = 1  
# G = nx.Graph()
# G.add_edges_from([(0,4),(0,5),(0,6),(0,7),(0,1),
#                   (1,4),(1,5),(1,6),(1,7),(1,2),
#                   (2,4),(2,5),(2,6),(2,7),
#                   (3,4),(3,5),(3,6),(3,7)])

# cuts = {}
# for i in range(n_iterations):
#     S = Simulation(G, p)
#     res = S.run()
#     # print(f"Total Energy Light Cones: {S.sum_of_expectations(res.x)}")   
#     G_new = G.copy()
#     constraints = RQAOA(G_new, p, res.x, {})
#     assignment = find_assignment(G, constraints)
#     cut = tuple(assignment[key] for key in sorted(assignment))
#     cuts[cut] = cuts.get(cut, 0) + 1
# # maxcut, count = max(cuts.items(), key= lambda x:x[1])
# # print(''.join(map(str, maxcut)), count)
# top3 = sorted(cuts.items(), key=lambda x: x[1], reverse=True)[:3]
# for cut, count in top3:
#     print(''.join(map(str, cut)), count)

In [ ]:
# n_iterations = 10
# p = 1  
# G = nx.Graph()

# mapping, G = generate_bipartite_ring_network(n=4, p=1, q=3)
# reversed_mapping = {mapping[key]: key for key in mapping}

# cuts = {}
# for i in range(n_iterations):
#     S = Simulation(G, p)
#     res = S.run()
#     G_new = G.copy()
#     constraints = RQAOA(G_new, p, res.x, {})
#     assignment = find_assignment(G, constraints)
#     assignment = {reversed_mapping[key]: assignment[key] for key in assignment}
#     # for key in sorted(assignment):
#     #     print(key, assignment[key])
#     cut = tuple(assignment[key] for key in sorted(assignment))
#     cuts[cut] = cuts.get(cut, 0) + 1
# top10 = sorted(cuts.items(), key=lambda x: x[1], reverse=True)[:10]
# for cut, count in top10:
#     print(''.join(map(str, cut)), count)

In [ ]:
    # def relabel_graph(G):
    #     old_nodes = list(G.nodes)
    #     mapping = {old: new for new, old in enumerate(old_nodes)}
    #     G_new = nx.relabel_nodes(G, mapping)
    #     return mapping, G_new

    # def run(self, G: nx.Graph, p: dict, angles: list, constraint):
    #     if len(G.nodes) == 1 or len(G.edges) == 0:
    #         return constraint
    #     mapping, G_new = relabel_graph(G)
    #     # new = mapping[old]
    #     H = graph_to_hamiltonian(nx.to_numpy_array(G_new), len(G_new.nodes()))    
    #     Q = QAOA(p, H) 
    #     magnitude = {}
    #     for (i,j) in G.edges:
    #         Z_iZ_j = ZZ(nx.to_numpy_array(G_new), mapping[i], mapping[j], len(G_new.nodes()))
    #         magnitude[(i, j)] = self.expectation(Q, Z_iZ_j, angles)
    #     (i,j), max_magn = max(magnitude.items(), key=lambda item: abs(item[1]))
    #     s = np.sign(max_magn) 
    #     constraint[(i, j)] = s
    #     G.remove_node(j)
    #     # print(f"remove:", {j})
    #     return self.run(G, p, angles, constraint)